## Metrics & formulas for batched LLM inference  

### Notation

| symbol | description |
|--------|-------------|
| $N$ | batch size (e.g. 100 prompts) |
| $i$ | index of a sample in the batch |
| $T_{\text{batch}}$ | wall-clock time to generate the **whole** batch (measured once) |
| $\text{tok}_i$ | number of generated **tokens** in sample $i$ |
| $\displaystyle\Sigma_{\text{tok}}=\sum_{j=1}^{N}\text{tok}_j$ | total tokens in the batch |
| $\text{sent}_i$ | number of **sentences** in sample $i$ |
| $\displaystyle\Sigma_{\text{sent}}=\sum_{j=1}^{N}\text{sent}_j$ | total sentences in the batch |

---

### 1&nbsp;· Average-token latency (ATL)

$$
\operatorname{ATL}_{\text{batch}}
    =\frac{T_{\text{batch}}}{\Sigma_{\text{tok}}}
    \quad\text{[seconds / token]}
$$

---

### 2&nbsp;· Per-sample generation latency (GL)

$$
\operatorname{GL}_i
    =\operatorname{ATL}_{\text{batch}}\;\text{tok}_i
    =\frac{\text{tok}_i}{\Sigma_{\text{tok}}}\;T_{\text{batch}}
    \quad\text{[seconds / sample]}
$$

Check:  $\displaystyle\sum_{i=1}^{N}\operatorname{GL}_i=T_{\text{batch}}$.

---

### 3 · Tokens per second (TPS)

Batch-level throughput:

$$
\text{TPS}_{\text{batch}}
  = \frac{1}{\text{ATL}_{\text{batch}}}
  = \frac{\Sigma_{\text{tok}}}{T_{\text{batch}}}
$$

*(units: tokens s⁻¹)*


### 4 · Sentences per second (SPS)

**Per-sample (row-level)**  

$$
\text{SPS}_i
  = \frac{\text{sent}_i}{\text{GL}_i}
  = \frac{\text{sent}_i}
         {\text{ATL}_{\text{batch}}\;\text{tok}_i}
$$

**Batch-level (one value per batch)**  

$$
\text{SPS}_{\text{batch}}
  = \frac{\Sigma_{\text{sent}}}{T_{\text{batch}}}
  = \frac{1}{N} \sum_{i=1}^{N} \text{SPS}_i
$$

---

### 5&nbsp;· Consistency checks (should hold after the fix)

$$
\sum_{i=1}^{N}\operatorname{GL}_i = T_{\text{batch}},\qquad
\frac{1}{N}\sum_{i=1}^{N}\operatorname{SPS}_i = \operatorname{SPS}_{\text{batch}},\qquad
\operatorname{TPS}_{\text{batch}}\;T_{\text{batch}} = \Sigma_{\text{tok}}
$$


In [31]:
import re
import pandas as pd
import numpy as np

data = pd.read_csv("/home/ubuntu/fast_llm_inference/results/experiment_1/vllm_gemma-2-2b-it_qa.csv")

In [ ]:
import re, argparse, pathlib
import pandas as pd, numpy as np

# ─── lightweight token / sentence counters ────────────────────────────
_tok_re  = re.compile(r"\S+")
_sent_re = re.compile(r"[.!?…]+")

def _tok_cnt(text: str) -> int:
    return len(_tok_re.findall(text))

def _sent_cnt(text: str) -> int:
    return max(len(_sent_re.findall(text)), 1)

# ─── main fixer ────────────────────────────────────────────────────────
def fix_metrics(df: pd.DataFrame, batch_size: int = 100) -> pd.DataFrame:
    """
    Fixes latency/throughput and overrides energy-per-unit columns in-place.
    Expects:
      - 'generated_answer'    : str
      - 'GL'                  : float (batch latency duplicated on every row)
      - 'Total Energy (Wh)'   : float (duplicated per row)
      - 'Energy per Token (J/token)'       : float (will be replaced)
      - 'Energy per Sentence (J/sentence)' : float (will be replaced)
    Returns the DataFrame with corrected:
      ATL, GL, TPS, SPS,
      Energy per Token (J/token),
      Energy per Sentence (J/sentence)
    """
    df = df.copy()

    # 1) derive token & sentence counts if missing -----------------------
    if "num_tokens" not in df.columns:
        df["num_tokens"] = df["generated_answer"].map(_tok_cnt)
    if "num_sentences" not in df.columns:
        df["num_sentences"] = df["generated_answer"].map(_sent_cnt)

    # 2) tag rows by batch -----------------------------------------------
    df["batch_id"] = (df.index // batch_size).astype(int)

    # 3) compute per-batch scalars ---------------------------------------
    df["batch_time_s"]      = df.groupby("batch_id")["GL"].transform("first")
    df["batch_tokens"]      = df.groupby("batch_id")["num_tokens"].transform("sum")
    df["batch_sentences"]   = df.groupby("batch_id")["num_sentences"].transform("sum")
    batch_energy_wh         = df.groupby("batch_id")["Total Energy (Wh)"].transform("first")

    # 4) correct latency & throughput ------------------------------------
    df["ATL"] = df["batch_time_s"] / df["batch_tokens"]      # seconds / token
    df["GL"]  = df["ATL"] * df["num_tokens"]                 # seconds / sample
    df["TPS"] = 1.0 / df["ATL"]                              # tokens / second
    df["SPS"] = df["num_sentences"] / df["GL"]               # sentences / second

    # 5) override energy-per-unit columns in-place -----------------------
    # convert Wh → J by multiplying by 3600
    df["Energy per Token (J/token)"]     = batch_energy_wh * 3600 / df["batch_tokens"]
    df["Energy per Sentence (J/sentence)"] = batch_energy_wh * 3600 / df["batch_sentences"]

    # 6) drop internal helpers -------------------------------------------
    df.drop(columns=["batch_id", "batch_time_s", "batch_tokens", "batch_sentences", "num_tokens", "num_sentences"], inplace=True)

    return df

In [46]:
fixed_data = fix_metrics(data, batch_size=100)

fixed_data

,prompt_length,prompt,generated_answer,reference_answer,TTFT,ATL,GL,TPS,SPS,Avg GPU Mem (MB),...,Avg GPU Util (%),Total Energy (Wh),Avg Power (W),Energy per Token (J/token),Energy per Sentence (J/sentence),Memory Usage (MB),Model Size (MB),Overhead (MB),exact_match,F1_score
0,1275,### SYSTEM\nYou are a question-answering assis...,Lothar de Maizière,"['Lothar de Maizière', 'Lothar de Maizière', '...",0.0283,0.009702,0.029107,103.069379,34.356460,21351.79,...,79.73,0.034697,56.71,0.550261,1.249092,21353.88,5007.298138,16346.581862,1,1.000000
1,753,### SYSTEM\nYou are a question-answering assis...,Complexity classes,"['complexity classes', 'complexity classes', '...",0.0283,0.009702,0.019404,103.069379,51.534689,21351.79,...,79.73,0.034697,56.71,0.550261,1.249092,21353.88,5007.298138,16346.581862,1,1.000000
2,1197,### SYSTEM\nYou are a question-answering assis...,GTE,['Telenet was incorporated in 1973 and started...,0.0283,0.009702,0.009702,103.069379,103.069379,21351.79,...,79.73,0.034697,56.71,0.550261,1.249092,21353.88,5007.298138,16346.581862,1,1.000000
3,1541,### SYSTEM\nYou are a question-answering assis...,Water flow,"['water flow through the body cavity', ""κτείς ...",0.0283,0.009702,0.019404,103.069379,51.534689,21351.79,...,79.73,0.034697,56.71,0.550261,1.249092,21353.88,5007.298138,16346.581862,0,0.571429
4,1747,### SYSTEM\nYou are a question-answering assis...,1705,"['12 May 1705', '1705', '12 May 1705']",0.0283,0.009702,0.009702,103.069379,103.069379,21351.79,...,79.73,0.034697,56.71,0.550261,1.249092,21353.88,5007.298138,16346.581862,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1157,### SYSTEM\nYou are a question-answering assis...,"1,600","['1,600 miles', '1,600', '1,600']",0.0276,0.009120,0.009120,109.650142,109.650142,21353.88,...,90.45,0.040963,68.52,0.624859,1.474668,21353.88,5007.298138,16346.581862,1,1.000000
496,2207,### SYSTEM\nYou are a question-answering assis...,Krasiński Palace Garden,"['Krasiński Palace Garden', 'Krasiński Palace ...",0.0276,0.009120,0.027360,109.650142,36.550047,21353.88,...,90.45,0.040963,68.52,0.624859,1.474668,21353.88,5007.298138,16346.581862,1,1.000000
497,1115,### SYSTEM\nYou are a question-answering assis...,214,"['489', '489', '489']",0.0276,0.009120,0.009120,109.650142,109.650142,21353.88,...,90.45,0.040963,68.52,0.624859,1.474668,21353.88,5007.298138,16346.581862,0,0.000000
498,1830,### SYSTEM\nYou are a question-answering assis...,Mnemiopsis leidyi,"['ctenophore Mnemiopsis leidyi', 'Mnemiopsis l...",0.0276,0.009120,0.018240,109.650142,54.825071,21353.88,...,90.45,0.040963,68.52,0.624859,1.474668,21353.88,5007.298138,16346.581862,1,1.000000


In [ ]:
#!/usr/bin/env python3
# merge_run_reports.py
# Usage: python merge_run_reports.py /home/ubuntu/fast_llm_inference/RQ1_merged_256/run_report

import sys
from pathlib import Path

import pandas as pd
import numpy as np

# ─────────────────────────────────────── config
RUN_DIR   = Path("/home/ubuntu/fast_llm_inference/RQ1_merged_256/run_report")
OUT_PATH  = RUN_DIR.parent / "merged_run_report.csv"

# All columns we want in the final file (order matters)
FINAL_COLS = [
    "model_name", "model_size_mb", "task", "scenario", "backend",
    "startup", "ttft_sec", "coldstart", "batch_size", "num_queries",
    "total_generation_time_s", "avg_gpu_mem_mb", "peak_gpu_mem_mb",
    "overhead_mb", "avg_gpu_util_pct", "peak_gpu_util_pct",
    "avg_cpu_util_pct", "peak_cpu_util_pct", "avg_ram_mb", "peak_ram_mb",
    "avg_power_w", "peak_power_w", "total_energy_wh",
    "avg_generation_time", "avg_tokens_generated", "avg_sentences_generated",
    "avg_ATL", "avg_GL", "avg_TPS", "avg_SPS",
    "avg_energy_per_token", "avg_energy_per_sentence"
]

# Quality‐metric columns we need for pick_quality()
QUALITY_COLS = ["avg_F1_score", "avg_AST_equal", "avg_ROUGE-1"]

def load_one(path: Path) -> pd.DataFrame:
    """Read one run_report CSV and normalize to FINAL_COLS + QUALITY_COLS schema."""
    df = pd.read_csv(path)

    # Ensure every expected column exists; missing ⇒ NaN
    for col in FINAL_COLS + QUALITY_COLS:
        if col not in df.columns:
            df[col] = np.nan

    # Keep only the final schema & order
    return df[FINAL_COLS + QUALITY_COLS]

# ─────────────────────────────────────── main
all_csvs = sorted(RUN_DIR.glob("*.csv"))
if not all_csvs:
    sys.exit(f"No CSVs found in {RUN_DIR}")

# Load & normalize each, then concatenate
merged = pd.concat([load_one(p) for p in all_csvs], ignore_index=True)

# ── write out ───────────────────────────────────────────────────────
merged.to_csv(OUT_PATH, index=False)
print(f"🚀  Merged {len(all_csvs)} files → {OUT_PATH}  ({len(merged)} rows)")

In [17]:
import pandas as pd

data = pd.read_csv("/home/ubuntu/fast_llm_inference/results/llama.cpp_gemma-2-2b-it-fp16.gguf_sql_1QPS_120s_server.csv")

In [18]:
data.columns

Index(['prompt_length', 'prompt', 'generated_answer', 'reference_answer',
       'queue_size', 'batch_size', 'wait_time', 'response_time',
       'scheduled_ts', 'start_ts', 'GL', 'ATL', 'TTFT', 'TPS', 'SPS',
       'Avg GPU Mem (MB)', 'Peak GPU Mem (MB)', 'Avg GPU Util (%)',
       'Peak GPU Util (%)', 'Total Energy (Wh)', 'Avg Power (W)',
       'Peak Power (W)', 'Energy per Token (J/token)',
       'Energy per Sentence (J/sentence)', 'Memory Usage (MB)',
       'Model Size (MB)', 'Overhead (MB)'],
      dtype='object')

In [19]:
data

,prompt_length,prompt,generated_answer,reference_answer,queue_size,batch_size,wait_time,response_time,scheduled_ts,start_ts,...,Avg GPU Util (%),Peak GPU Util (%),Total Energy (Wh),Avg Power (W),Peak Power (W),Energy per Token (J/token),Energy per Sentence (J/sentence),Memory Usage (MB),Model Size (MB),Overhead (MB)
0,984,### SYSTEM\nYou are a SQL query generation ass...,SELECT AirportCode FROM airports ORDER BY COUN...,SELECT T1.AirportCode FROM AIRPORTS AS T1 JOIN...,1,1,0.0007,0.4945,1.3042,1.3048,...,40.00,86,0.005681,41.41,47.83,2.045035,20.450349,7005.88,4992.689056,2013.190944
1,952,### SYSTEM\nYou are a SQL query generation ass...,SELECT COUNT(*) FROM Student AS s JOIN Has_Pet...,SELECT count(*) FROM student AS T1 JOIN has_pe...,1,1,0.0165,1.5940,2.2943,2.3107,...,71.57,87,0.025997,59.33,73.20,2.836066,18.718035,7005.88,4992.689056,2013.190944
2,1984,### SYSTEM\nYou are a SQL query generation ass...,```sql,SELECT avg(transcript_date) FROM Transcripts,2,2,1.3742,2.6147,2.5643,3.9385,...,78.10,91,0.023225,67.40,73.20,83.609322,83.609322,7009.88,4992.689056,2017.190944
3,1084,### SYSTEM\nYou are a SQL query generation ass...,```sql,SELECT name FROM battle WHERE bulgarian_comman...,2,2,0.7772,2.0177,3.1613,3.9385,...,78.10,91,0.023225,67.40,73.20,83.609322,83.609322,7009.88,4992.689056,2017.190944
4,1385,### SYSTEM\nYou are a SQL query generation ass...,```sql,"SELECT first_name , last_name FROM players WH...",1,1,0.0426,1.5931,6.9972,7.0398,...,64.65,85,0.026734,62.07,72.98,96.243709,96.243709,7009.88,4992.689056,2017.190944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,934,### SYSTEM\nYou are a SQL query generation ass...,"SELECT PetType, AVG(weight) FROM Pets GROUP BY...","SELECT avg(weight) , pettype FROM pets GROUP ...",1,1,0.0002,0.4529,108.6052,108.6055,...,0.00,0,0.004840,38.49,40.44,2.178195,17.425557,7011.88,4992.689056,2019.190944
99,1198,### SYSTEM\nYou are a SQL query generation ass...,```sql,SELECT T1.series_name FROM TV_Channel AS T1 JO...,1,1,0.0367,1.3120,113.5299,113.5666,...,67.91,86,0.020092,56.72,68.47,72.331712,72.331712,7011.88,4992.689056,2019.190944
100,1615,### SYSTEM\nYou are a SQL query generation ass...,```sql,"SELECT T1.date_of_treatment , T2.first_name F...",1,1,0.8646,1.9765,114.1040,114.9686,...,74.30,88,0.021860,70.77,71.83,78.697108,78.697108,7011.88,4992.689056,2019.190944
101,1190,### SYSTEM\nYou are a SQL query generation ass...,```sql,"select t1.id , t1.maker from car_makers as t1...",1,1,0.0169,2.2753,116.6142,116.6311,...,70.71,90,0.039308,62.66,73.00,141.509054,141.509054,7011.88,4992.689056,2019.190944


In [15]:
import sys
sys.path.append("/home/ubuntu/fast_llm_inference") 

from benchmark.tasks.summarization import SummarizationTask

st = SummarizationTask()

data = st.generate_prompts(256)

In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import os

import sys
sys.path.append("/home/ubuntu/fast_llm_inference") 

from benchmark.tasks.summarization import SummarizationTask

st = SummarizationTask()

data = st.generate_prompts(512)

# Prevent tokenizers or HF internals from using GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disables GPU access for subprocesses

# Load tokenizer (runs on CPU by default)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-32B-Instruct", trust_remote_code=True)

# Prepare prompt DataFrame
df = pd.DataFrame(data[0], columns=["prompt"])
df["tokens"] = df["prompt"].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=False)))
df["words"] = df["prompt"].apply(lambda x: len(x.split()))

# Filter prompts based on context window constraint
max_input_tokens = 1200 
df_filtered = df[df["tokens"] <= max_input_tokens].reset_index(drop=True)

print(f"{len(df_filtered)} / {len(df)} prompts retained under {max_input_tokens} tokens.")

301 / 512 prompts retained under 1200 tokens.


In [23]:
from transformers import AutoTokenizer
import pandas as pd
import os

import sys
sys.path.append("/home/ubuntu/fast_llm_inference") 

from benchmark.tasks.summarization import SummarizationTask

st = SummarizationTask()

data = st.generate_prompts(256, max_words=1360)

# Prevent tokenizers or HF internals from using GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disables GPU access for subprocesses

# Load tokenizer (runs on CPU by default)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4", trust_remote_code=True)

# Prepare prompt DataFrame
df = pd.DataFrame(data[0], columns=["prompt"])
df["tokens"] = df["prompt"].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=False)))
df["words"] = df["prompt"].apply(lambda x: len(x.split()))

# Filter prompts based on context window constraint
max_input_tokens = 1970
df_filtered = df[df["tokens"] <= max_input_tokens].reset_index(drop=True)

print(f"{len(df_filtered)} / {len(df)} prompts retained under {max_input_tokens} tokens.")

256 / 256 prompts retained under 1970 tokens.


In [24]:
df['tokens'].max()

1851

In [31]:
df['tokens'].mean() / df['words'].mean()

1.3406135064833975

In [2]:
from huggingface_hub import HfApi, hf_hub_download, list_repo_files
import json
from pathlib import Path
from typing import Tuple

api = HfApi()

def hf_model_size(repo_id: str, revision: str | None = None) -> Tuple[int, float, float]:
    """
    Return (bytes, mebibytes, gibibytes) occupied by `repo_id` on Hugging Face.

    Parameters
    ----------
    repo_id   : "owner/model_name" on HF Hub.
    revision  : branch / tag / commit hash (default: main).

    Examples
    --------
    >>> hf_model_size("Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4")
    (15321596485, 14609.49, 14.27)
    """
    # 1) Try the fast path ─ total_size inside *.index.json
    try:
        index_path = hf_hub_download(
            repo_id, "model.safetensors.index.json",
            repo_type="model", revision=revision, local_dir="/tmp", local_dir_use_symlinks=False
        )
        with open(index_path, "r", encoding="utf-8") as f:
            total = json.load(f).get("metadata", {}).get("total_size")
            if isinstance(total, int) and total > 0:
                mib = total / (1024**2)
                gib = total / (1024**3)
                return mib, gib
    except Exception:
        pass  # fall back to listing files

    # 2) Fallback ─ sum all weight files
    weight_exts = {".bin", ".safetensors", ".pt", ".pth"}
    size_sum = 0
    for file_info in api.list_repo_files(repo_id, revision=revision, repo_type="model"):
        if Path(file_info).suffix in weight_exts:
            # head() gives the object metadata without downloading
            hf_obj = api.head(repo_id, file_info, revision=revision, repo_type="model")
            size_sum += hf_obj.size

    mib = size_sum / (1024**2)
    gib = size_sum / (1024**3)
    return mib, gib

if __name__ == "__main__":
    model_id = "google/gemma-2-9b-it"
    print(hf_model_size(model_id))

(17627.1552734375, 17.21401882171631)


In [36]:
import random
import json
import sqlite3
from functools import lru_cache
from pathlib import Path
from typing import Union, Tuple, List, Dict

from datasets import load_dataset
from sqlglot import parse_one
import sqlparse

LOOKUP_DIR = Path("/home/ubuntu/fast_llm_inference/benchmark/lookup")

class SQLTask:
    """SQL generation task for the Spider dataset with optional execution accuracy."""

    def __init__(self,
                 tables_path: Union[str, Path, None] = None,
                 db_root: Union[str, Path, None] = None,
                 seed: int = 42):
        random.seed(seed)
        self.dataset = list(load_dataset("spider", split="train"))

        self.tables_path: Path = Path(tables_path or LOOKUP_DIR / "tables.json")
        if not self.tables_path.exists():
            raise FileNotFoundError(f"Spider tables.json not found at {self.tables_path}.")

        # Try to find or use given database path
        if db_root is not None:
            db_root_path = Path(db_root).expanduser().resolve()
        else:
            db_root_path = self.tables_path.parent / "database"

        if db_root_path.is_dir():
            self.db_root = db_root_path
            self.execution_enabled = True
        else:
            self.db_root = None
            self.execution_enabled = False
            print("[SQLTask] Warning: database/ folder not found. Execution accuracy will be disabled.")

    @lru_cache(maxsize=None)
    def _conn(self, db_id: str) -> sqlite3.Connection:
        if not self.execution_enabled:
            raise RuntimeError("Execution not available: no database folder found.")
        db_file = self.db_root / db_id / f"{db_id}.sqlite"
        if not db_file.exists():
            raise FileNotFoundError(f"SQLite DB for db_id='{db_id}' not found at {db_file}.")
        return sqlite3.connect(f"file:{db_file}?immutable=1", uri=True, timeout=1.0, isolation_level=None)

    def _exec(self, db_id: str, query: str) -> set:
        if not self.execution_enabled:
            raise RuntimeError("Execution disabled (no database folder).")
        cur = self._conn(db_id).cursor()
        cur.execute(query.strip().rstrip(";"))
        rows = cur.fetchall()
        return {tuple(r) for r in rows}

    def generate_prompts(self, num_examples: int = 100) -> Tuple[List[str], List[str], List[str]]:
        sampled = random.sample(self.dataset, num_examples)
        prompts    = [self._sql_prompt(ex) for ex in sampled]
        references = [ex["query"]         for ex in sampled]
        db_ids     = [ex["db_id"]         for ex in sampled]
        return prompts, references, db_ids

    def _table_columns(self, db_id: str) -> str:
        with open(self.tables_path, "r", encoding="utf-8") as f:
            tables_json = json.load(f)

        for db in tables_json:
            if db["db_id"] == db_id:
                table_names = db["table_names_original"]
                column_info = db["column_names_original"]
                table_columns = {t: [] for t in table_names}
                for table_idx, col_name in column_info:
                    if col_name == "*":
                        continue
                    table_columns[table_names[table_idx]].append(col_name)
                return "\n".join(
                    f"Table '{tbl}': columns = {', '.join(cols)}" for tbl, cols in table_columns.items()
                )

        return f"No schema found for db_id='{db_id}'."

    def _sql_prompt(self, example: Dict) -> str:
        question = example["question"]
        column_context = self._table_columns(example["db_id"])

        system_message = (
            "You are a SQL query generation assistant.  Given a natural language "
            "question and the database schema, produce only the corresponding SQL "
            "statement ending with a semicolon.  No explanations.\n\n"
        )

        demo_block = (
            "### EXAMPLES\n"
            "Question:\n"
            "How many heads of the departments are older than 56?\n\n"
            "Tables in the database:\n"
            "Table 'department': columns = Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees\n"
            "Table 'head':       columns = head_ID, name, born_state, age\n"
            "Table 'management': columns = department_ID, head_ID, temporary_acting\n\n"
            "SQL:\n"
            "SELECT COUNT(*) FROM head WHERE age > 56;\n\n"
        )

        instruction = "### INSTRUCTION\nGenerate the SQL statement that answers the question.\n\n"
        input_block = (
            "### INPUT\n"
            f"Question:\n{question}\n\n"
            "Tables in the database:\n"
            f"{column_context}\n\n"
        )
        output_header = "### OUTPUT\nSQL:\n"

        return (
            f"### SYSTEM\n{system_message}\n"
            f"{demo_block}"
            f"{instruction}"
            f"{input_block}"
            f"{output_header}"
        )

    def quality_metrics(self,
                        generated: str,
                        reference: str,
                        db_id: str | None = None) -> dict[str, float]:

        def ast_equal(sql1: str, sql2: str) -> int:
            try:
                return int(parse_one(sql1.lower()) == parse_one(sql2.lower()))
            except Exception:
                return 0

        metrics = {"AST_equal": ast_equal(generated, reference)}

        if db_id is not None and self.execution_enabled:
            try:
                # 1️⃣ Run full reference to get the “correct” result set
                ref_res = self._exec(db_id, reference)

                # 2️⃣ Split generated SQL into individual statements
                stmts = [s.strip() for s in sqlparse.split(generated) if s.strip()]

                # 3️⃣ Execute each one; if any matches ref_res, we’re good
                exec_correct = 0.0
                for stmt in stmts:
                    try:
                        gen_res = self._exec(db_id, stmt)
                        if gen_res == ref_res:
                            exec_correct = 1.0
                            break
                    except Exception:
                        # ignore bad statements
                        continue

                metrics["Exec_accuracy"] = exec_correct

            except Exception as e:
                print(f"[warn] Execution error for db_id='{db_id}': {e}")
                metrics["Exec_accuracy"] = 0.0

        return metrics

In [16]:
task = SQLTask(db_root="/home/ubuntu/fast_llm_inference/benchmark/lookup/database", tables_path="/home/ubuntu/fast_llm_inference/benchmark/lookup/tables.json")
prompts, refs, db_ids = task.generate_prompts(num_examples=256)

In [32]:
import os
import re
import shutil
from glob import glob
from collections import defaultdict

# ─── 1️⃣ Setup ─────────────────────────────────────────────────────
base_dir = "/home/ubuntu/fast_llm_inference"
datasets = ["RQ1", "RQ1_rerun", "RQ2_256", "RQ2_rerun"]
subdirs = ["details", "readings", "run_report"]
output_dir = os.path.join(base_dir, "RQ1_merged_256")

# Create clean target dir
for subdir in subdirs:
    os.makedirs(os.path.join(output_dir, subdir), exist_ok=True)

# ─── 2️⃣ Normalize filenames (strip hash) ──────────────────────────
def normalize_filename(path, is_reading=False):
    fname = os.path.basename(path)
    if is_reading:
        fname = fname.replace("ts_", "")
    base = re.sub(r"(_[a-f0-9]{8})?\.csv$", "", fname)
    return base

# ─── 3️⃣ Find all CSVs and group by normalized name ────────────────
file_registry = {s: defaultdict(list) for s in subdirs}

for dataset in datasets:
    for subdir in subdirs:
        dir_path = os.path.join(base_dir, dataset, subdir)
        is_reading = (subdir == "readings")
        for path in glob(f"{dir_path}/*.csv"):
            key = normalize_filename(path, is_reading)
            file_registry[subdir][key].append(path)

# ─── 4️⃣ Copy first file per key to merged folder ──────────────────
for subdir in subdirs:
    merged_subdir = os.path.join(output_dir, subdir)
    print(f"\n📂 Merging unique files from '{subdir}'")
    for key, files in file_registry[subdir].items():
        src = files[0]  # take the first file regardless of duplicates
        dst = os.path.join(merged_subdir, os.path.basename(src))
        shutil.copy(src, dst)

        if len(files) > 1:
            print(f"⚠️ Duplicate detected for '{key}' — kept: {os.path.basename(src)}")



📂 Merging unique files from 'details'
⚠️ Duplicate detected for 'vllm_gemma-2-9b-it_sql_batch_bs16' — kept: vllm_gemma-2-9b-it_sql_batch_bs16_97331b63.csv
⚠️ Duplicate detected for 'vllm_Qwen2.5-7B-Instruct_sql_batch_bs16' — kept: vllm_Qwen2.5-7B-Instruct_sql_batch_bs16_31034d81.csv
⚠️ Duplicate detected for 'vllm_gemma-2-9b-it_qa_batch_bs16' — kept: vllm_gemma-2-9b-it_qa_batch_bs16_34b1c6f1.csv
⚠️ Duplicate detected for 'vllm_Mistral-7B-Instruct-v0.3_qa_batch_bs16' — kept: vllm_Mistral-7B-Instruct-v0.3_qa_batch_bs16_e95403f7.csv
⚠️ Duplicate detected for 'vllm_Mistral-7B-Instruct-v0.3_summarization_batch_bs16' — kept: vllm_Mistral-7B-Instruct-v0.3_summarization_batch_bs16_b840bb9a.csv
⚠️ Duplicate detected for 'vllm_Qwen2.5-7B-Instruct_summarization_batch_bs16' — kept: vllm_Qwen2.5-7B-Instruct_summarization_batch_bs16_b484770d.csv
⚠️ Duplicate detected for 'vllm_Qwen2.5-7B-Instruct_qa_batch_bs16' — kept: vllm_Qwen2.5-7B-Instruct_qa_batch_bs16_473ad563.csv
⚠️ Duplicate detected for 'v

In [33]:
import re
from pathlib import Path
import yaml
import pandas as pd

DETAILS_DIR = Path("/home/ubuntu/fast_llm_inference/RQ1_merged_256/details")

CONFIG_MODELS = """
# Qwen2.5
- Qwen/Qwen2.5-1.5B-Instruct
- Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-3B-Instruct
- Qwen/Qwen2.5-3B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-3B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-7B-Instruct
- Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-14B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4

# Mistral
- mistralai/Mistral-7B-Instruct-v0.3
- marinarosell/Mistral-7B-Instruct-v0.3-GPTQ-8bit-gs128
- marinarosell/Mistral-7B-Instruct-v0.3-GPTQ-4bit-gs128
- dwetzel/Mistral-Small-24B-Instruct-2501-GPTQ-INT4

# Gemma-2
- google/gemma-2-2b-it
- marcinbrzezanski/gemma-2-2b-it-gptq
- RedHatAI/gemma-2-2b-it-quantized.w8a16
- google/gemma-2-9b-it
- shuyuej/gemma-2-9b-it-GPTQ
- RedHatAI/gemma-2-9b-it-quantized.w8a16
- shuyuej/gemma-2-27b-it-GPTQ

# LLaMA 3.x
- meta-llama/Llama-3.2-1B-Instruct
- clowman/Llama-3.2-1B-Instruct-GPTQ-Int4
- clowman/Llama-3.2-1B-Instruct-GPTQ-Int8
- meta-llama/Llama-3.2-3B-Instruct
- clowman/Llama-3.2-3B-Instruct-GPTQ-Int8
- clowman/Llama-3.2-3B-Instruct-GPTQ-Int4
- meta-llama/Llama-3.1-8B-Instruct
- clowman/Llama-3.1-8B-Instruct-GPTQ-Int4
- clowman/Llama-3.1-8B-Instruct-GPTQ-Int8
"""

# ---------------------------------------------------------------------------
expected_models = [m.split("/")[-1] for m in yaml.safe_load(CONFIG_MODELS)]
tasks = ["qa", "sql", "summarization"]

# 1️⃣  Scan all CSVs
existing_files = list(DETAILS_DIR.glob("*.csv"))

# helper → strip “vllm_” prefix and trailing “_<8-hex>.csv”
def normalize(fname: str) -> str:
    core = re.sub(r"^vllm_", "", fname)
    return re.sub(r"_[a-f0-9]{8}\.csv$", "", core)

# Build presence map and duplicate map
present = {m: {t: False for t in tasks} for m in expected_models}
dupes: dict[str, list[str]] = {}

for f in existing_files:
    norm = normalize(f.name)                       # e.g. Llama-..._qa_batch_bs16
    dupes.setdefault(norm, []).append(f.name)      # collect for duplicate check

    parts = norm.split("_")
    if len(parts) < 4:
        continue
    model_id = "_".join(parts[:-3])
    task      = parts[-3]
    if model_id in present and task in present[model_id]:
        present[model_id][task] = True

# 2️⃣  “Which runs are still missing?”
records = []
for model in expected_models:
    st = present[model]
    records.append(
        {"Model": model, "qa": st["qa"], "sql": st["sql"],
         "summarization": st["summarization"], "Complete": all(st.values())}
    )
missing_df = pd.DataFrame(records).sort_values("Model")

# 3️⃣  “Which runs are duplicated (same model+task, different hashes)?”
dup_records = [
    {"Normalized name": k, "Count": len(v), "Files": " | ".join(v)}
    for k, v in dupes.items() if len(v) > 1
]

if dup_records:
    dupes_df = pd.DataFrame(dup_records).sort_values("Count", ascending=False)
else:
    dupes_df = pd.DataFrame(columns=["Normalized name", "Count", "Files"])


# 4️⃣  Show / save results
print("\n=== Missing (incomplete) runs ===")
print(missing_df[~missing_df["Complete"]])

print("\n=== Duplicate CSVs (hash-agnostic) ===")
print(dupes_df if not dupes_df.empty else "No duplicates 🎉")


=== Missing (incomplete) runs ===
Empty DataFrame
Columns: [Model, qa, sql, summarization, Complete]
Index: []

=== Duplicate CSVs (hash-agnostic) ===
No duplicates 🎉


In [34]:
#!/usr/bin/env python3
# merge_benchmark_details.py
# Usage:  python merge_benchmark_details.py /home/ubuntu/fast_llm_inference/RQ1_merged/details

import re
import sys
from pathlib import Path

import pandas as pd
import numpy as np

# ─────────────────────────────────────── config
DETAILS_DIR     = Path("/home/ubuntu/fast_llm_inference/RQ1_merged_256/details")
OUT_PATH        = DETAILS_DIR.parent / "merged_details.csv"

# All columns we want in the final file (order matters)
FINAL_COLS = [
    "backend", "model", "task",
    "generated_answer", "reference_answer",
    "generation_time", "tokens_generated", "sentences_generated",
    "ATL", "GL", "TPS", "SPS",
    "energy_per_token", "energy_per_sentence",
    "exact_match", "F1_score",
    "AST_equal", "Normalized_equal",
    "ROUGE-1", "ROUGE-2", "ROUGE-L"
]

# Regex to pull <backend>, <model>, <task> from the filename
PAT = re.compile(
    r"""^(?P<backend>[^_]+)_           # vllm, tgi …
        (?P<model>.+?)_                # greedy until _<task>_
        (?P<task>qa|sql|summarization) # capture task
        _[^/]*\.csv$                   # rest is scenario, hash …
    """, re.X | re.I)

def load_one(path: Path) -> pd.DataFrame:
    """Read a single details-CSV and normalise its columns."""
    m = PAT.match(path.name)
    if not m:
        raise ValueError(f"Cannot parse '{path.name}'")

    df = pd.read_csv(path)

    # Drop prompt – we don't need it in the merged file
    df = df.drop(columns=[c for c in df.columns if c.lower() == "prompt"], errors="ignore")

    # Inject backend / model / task from filename
    for k, v in m.groupdict().items():
        df[k] = v

    # Make sure every expected column exists; missing ⇒ NaN
    for col in FINAL_COLS:
        if col not in df.columns:
            df[col] = np.nan

    # Keep only the final schema & order
    return df[FINAL_COLS]

# ─────────────────────────────────────── main
all_csvs = sorted(DETAILS_DIR.glob("*.csv"))
if not all_csvs:
    sys.exit(f"No CSVs found in {DETAILS_DIR}")

merged = pd.concat([load_one(p) for p in all_csvs], ignore_index=True)
merged.to_csv(OUT_PATH, index=False)
print(f"🚀  Merged {len(all_csvs)} files → {OUT_PATH}  ({len(merged)} rows)")


🚀  Merged 96 files → /home/ubuntu/fast_llm_inference/RQ1_merged_256/merged_details.csv  (24576 rows)


In [37]:
sql_task = SQLTask(
    db_root="/home/ubuntu/fast_llm_inference/benchmark/lookup/database",
    tables_path="/home/ubuntu/fast_llm_inference/benchmark/lookup/tables.json"
)
df_sql_id = pd.DataFrame(sql_task.generate_prompts(num_examples=256)[1:])
df_sql_id = df_sql_id.T            # flips rows↔columns
df_sql_id.columns = ["reference", "db_id"]

In [38]:
csv_path = "/home/ubuntu/fast_llm_inference/RQ1_merged_256/merged_details.csv"
merged = pd.read_csv(csv_path)

In [39]:
import numpy as np
import pandas as pd

# 1️⃣  Build a mapping:   reference → db_id
ref2db = dict(zip(df_sql_id["reference"], df_sql_id["db_id"]))

# 2️⃣  Initialise the column (optional; the map will create NaNs automatically)
merged["db_id"] = np.nan

# 3️⃣  Fill db_id only for the SQL rows
sql_mask = merged["task"].str.lower() == "sql"
merged.loc[sql_mask, "db_id"] = merged.loc[sql_mask, "reference_answer"].map(ref2db)

/tmp/ipykernel_3838/1186527573.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['music_2' 'insurance_fnol' 'bike_1' ... 'music_1' 'cre_Doc_Tracking_DB'
 'baseball_1']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged.loc[sql_mask, "db_id"] = merged.loc[sql_mask, "reference_answer"].map(ref2db)


In [40]:
import pandas as pd

# 1️⃣ Mask SQL rows
sql_mask = merged["task"].str.lower() == "sql"

# 2️⃣ Run quality_metrics on each SQL row, expand dict → columns
metrics_df = merged.loc[sql_mask].apply(
    lambda row: pd.Series(
        sql_task.quality_metrics(
            generated=row["generated_answer"],
            reference=row["reference_answer"],
            db_id=row["db_id"]
        )
    ),
    axis=1
)

# 3️⃣ Assign those metric-columns back into merged
merged.loc[sql_mask, metrics_df.columns] = metrics_df

In [41]:
merged[(merged['model'] == "Mistral-Small-24B-Instruct-2501-GPTQ-INT4") & (merged['task'] == "sql")]["Exec_accuracy"].mean()

0.5234375

In [43]:
merged[(merged['model'] == "gemma-2-9b-it") & (merged['task'] == "sql")]["Exec_accuracy"].mean()

0.6015625

In [44]:
merged[(merged['model'] == "gemma-2-27b-it-GPTQ") & (merged['task'] == "sql")]["Exec_accuracy"].mean()

0.6484375

In [2]:
import pandas as pd
import os

# ─── 1️⃣ Load df_report ──────────────────────────────────────────
df_report = pd.read_csv("/home/ubuntu/fast_llm_inference/RQ1_merged_256/merged_run_report_info.csv")
df_report["gpu_util_pct"] = None
df_report["gpu_util_pct_ci95"] = None

# ─── 2️⃣ File listing ────────────────────────────────────────────
readings_dir = "/home/ubuntu/fast_llm_inference/RQ1_merged_256/readings"
files = [f for f in os.listdir(readings_dir) if f.endswith(".csv") and f.startswith("ts_")]

matched, unmatched, skipped = 0, 0, 0

# ─── 3️⃣ Process each reading file ───────────────────────────────
for fname in files:
    path = os.path.join(readings_dir, fname)
    stem = fname.removesuffix(".csv")

    # Remove ts_ prefix
    if not stem.startswith("ts_"):
        skipped += 1
        continue
    stem = stem[3:]

    parts = stem.split("_")

    if len(parts) < 5:
        print(f"❌ Too few parts in filename: {fname}")
        skipped += 1
        continue

    # Extract metadata
    backend  = parts[0].strip().lower()
    model = parts[1].strip()
    task     = parts[2].strip().lower()
    scenario = parts[3].strip().lower()

    # Validate
    valid_tasks = {"sql", "qa", "summarization"}
    valid_scenarios = {"single", "batch", "server"}
    valid_backends = {"vllm", "tgi", "sglang", "lmdeploy"}

    if task not in valid_tasks or scenario not in valid_scenarios or backend not in valid_backends:
        print(f"⚠️ Unknown task or scenario in {fname}")
        skipped += 1
        continue

    # Read readings file
    try:
        df_reading = pd.read_csv(path)
    except Exception as e:
        print(f"❌ Could not read {fname}: {e}")
        skipped += 1
        continue

    if "gpu_util_pct" not in df_reading.columns:
        print(f"⚠️ Skipping {fname} — 'gpu_util_pct' missing")
        skipped += 1
        continue

    # Compute mean and CI95
    gpu_vals = df_reading["gpu_util_pct"].dropna()
    mean_val = gpu_vals.mean()
    std_val  = gpu_vals.std()
    n_val    = gpu_vals.count()
    ci95_val = 1.96 * std_val / (n_val ** 0.5) if n_val > 1 else 0.0

    # Find corresponding row
    mask = (
        (df_report["model_name"] == model)
        & (df_report["task"].str.lower() == task)
        & (df_report["scenario"].str.lower() == scenario)
        & (df_report["backend"].str.lower() == backend)
    )

    if mask.sum() == 1:
        df_report.loc[mask, "gpu_util_pct"] = mean_val
        df_report.loc[mask, "gpu_util_pct_ci95"] = ci95_val
        matched += 1
    elif mask.sum() == 0:
        print(f"❌ No match for model={model}, task={task}, scenario={scenario}, backend={backend}")
        unmatched += 1
    else:
        print(f"⚠️ Multiple matches for model={model}, task={task}, scenario={scenario}, backend={backend}")
        unmatched += 1

# ─── 4️⃣ Save updated report ─────────────────────────────────────
out_path = "/home/ubuntu/fast_llm_inference/RQ1_merged_256/merged_run_report_info_with_gpu_ci95.csv"
df_report.to_csv(out_path, index=False)

# ─── 5️⃣ Summary ─────────────────────────────────────────────────
print(f"✅ Done. Saved: {out_path}")
print(f"Matched:   {matched}")
print(f"Unmatched: {unmatched}")
print(f"Skipped:   {skipped}")

✅ Done. Saved: /home/ubuntu/fast_llm_inference/RQ1_merged_256/merged_run_report_info_with_gpu_ci95.csv
Matched:   96
Unmatched: 0
Skipped:   0


In [10]:
model_include = [
    "Mistral-7B-Instruct-v0.3", "Mistral-Small-24B-Instruct-2501-GPTQ-INT4",
    "gemma-2-9b-it", "gemma-2-27b-it-GPTQ",
    "Qwen2.5-7B-Instruct", "Qwen2.5-14B-Instruct-GPTQ-Int8", "Qwen2.5-32B-Instruct-GPTQ-Int4"
]

filtered_sizes = df_report[df_report['model_name'].isin(model_include)][['model_name', 'model_size_mb']]

filtered_sizes

,model_name,model_size_mb
33,Mistral-7B-Instruct-v0.3,13824.507812
34,Mistral-7B-Instruct-v0.3,13824.507812
35,Mistral-7B-Instruct-v0.3,13824.507812
36,Mistral-Small-24B-Instruct-2501-GPTQ-INT4,13492.045288
37,Mistral-Small-24B-Instruct-2501-GPTQ-INT4,13492.045288
38,Mistral-Small-24B-Instruct-2501-GPTQ-INT4,13492.045288
51,Qwen2.5-14B-Instruct-GPTQ-Int8,15875.072266
52,Qwen2.5-14B-Instruct-GPTQ-Int8,15875.072266
53,Qwen2.5-14B-Instruct-GPTQ-Int8,15875.072266
54,Qwen2.5-32B-Instruct-GPTQ-Int4,18447.634766


In [ ]:
import sys
from pathlib import Path
from transformers import AutoTokenizer

# ─── Adjust path for imports ───
sys.path.append(str(Path().resolve().parent))

from benchmark.tasks.summarization import SummarizationTask
from benchmark.tasks.sql import SQLTask
from benchmark.tasks.qa import QATask

# ─── Load tokenizer ───
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct", use_fast=True)

# ─── Define a helper to compute avg token length ───
def average_tokens(prompts, tokenizer):
    return sum(len(tokenizer(p, add_special_tokens=False)["input_ids"]) for p in prompts) / len(prompts)

# ─── Summarization ───
st = SummarizationTask()
sum_prompts = st.generate_prompts(256)[0]
avg_sum = average_tokens(sum_prompts, tokenizer)

# ─── SQL ───
sqlt = SQLTask()
sql_prompts = sqlt.generate_prompts(256)[0]
avg_sql = average_tokens(sql_prompts, tokenizer)

# ─── QA ───
qat = QATask()
qa_prompts = qat.generate_prompts(256)[0]
avg_qa = average_tokens(qa_prompts, tokenizer)

# ─── Print results ───
print(f"Average tokens per prompt:")
print(f"  • Summarization: {avg_sum:.2f}")
print(f"  • SQL:           {avg_sql:.2f}")
print(f"  • QA:            {avg_qa:.2f}")

Average tokens per prompt:
  • Summarization: 1166.49
  • SQL:           326.90
  • QA:            311.30


In [3]:
import pandas as pd
from pathlib import Path

# Path to the folder with CSVs
data_dir = Path("/home/ubuntu/fast_llm_inference/RQ3/details")
all_files = list(data_dir.glob("*.csv"))

# Filter for *_single_*.csv files
single_files = [f for f in all_files if "_single_" in f.stem]

# Load and annotate each CSV
df_list = []
for f in single_files:
    parts = f.stem.split("_")
    if len(parts) < 4:
        continue
    engine = parts[0]
    model = parts[1]
    task = parts[2]
    use_case = parts[3]

    df = pd.read_csv(f)
    df["engine"] = engine
    df["model"] = model
    df["task"] = task
    df["use_case"] = use_case
    df_list.append(df)

# Concatenate into one DataFrame
merged_single_df = pd.concat(df_list, ignore_index=True)

merged_single_df.to_csv("/home/ubuntu/fast_llm_inference/RQ3/merged_single_details.csv", index=False)

merged_single_df.columns

Index(['prompt', 'generated_answer', 'reference_answer', 'generation_time',
       'tokens_generated', 'sentences_generated', 'ATL', 'GL', 'TPS', 'SPS',
       'energy_per_token', 'energy_per_sentence', 'ROUGE-1', 'ROUGE-2',
       'ROUGE-L', 'engine', 'model', 'task', 'use_case', 'AST_equal',
       'exact_match', 'F1_score'],
      dtype='object')

In [8]:
# Re-import necessary modules due to code execution environment reset
import pandas as pd
from pathlib import Path

# Re-define path and find all CSVs again
data_dir = Path("/home/ubuntu/fast_llm_inference/RQ3/details")
all_files = list(data_dir.glob("*.csv"))

# Filter for *_batch_*.csv files
batch_files = [f for f in all_files if "_batch_" in f.stem]

# Load and annotate each CSV
batch_df_list = []
for f in batch_files:
    parts = f.stem.split("_")
    if len(parts) < 4:
        continue
    engine = parts[0]
    model = parts[1]
    task = parts[2]
    use_case = parts[3]
    batch_size = parts[4]

    df = pd.read_csv(f)
    df["engine"] = engine
    df["model"] = model
    df["task"] = task
    df["use_case"] = use_case
    df["batch_size"] = batch_size
    batch_df_list.append(df)

# Concatenate into one DataFrame
merged_batch_df = pd.concat(batch_df_list, ignore_index=True)

merged_batch_df.to_csv("/home/ubuntu/fast_llm_inference/RQ3/merged_batch_details.csv", index=False)

merged_batch_df


,prompt,generated_answer,reference_answer,generation_time,tokens_generated,sentences_generated,ATL,GL,TPS,SPS,...,engine,model,task,use_case,batch_size,ROUGE-1,ROUGE-2,ROUGE-L,exact_match,F1_score
0,### SYSTEM\nYou are a SQL query generation ass...,"SELECT Band.Firstname, Band.Lastname FROM Perf...","SELECT T2.firstname , T2.lastname FROM Perfor...",0.185022,13,2,0.014232,0.185022,70.26,10.81,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
1,### SYSTEM\nYou are a SQL query generation ass...,SELECT policy_type_code FROM Available_Policies,SELECT DISTINCT t3.policy_type_code FROM custo...,0.185022,12,1,0.015419,0.185022,64.86,5.40,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
2,### SYSTEM\nYou are a SQL query generation ass...,"SELECT station.id, station.name","SELECT DISTINCT T1.id , T1.name FROM station ...",0.185022,9,2,0.020558,0.185022,48.64,10.81,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
3,### SYSTEM\nYou are a SQL query generation ass...,SELECT customer_name FROM Customers WHERE cust...,SELECT customer_name FROM customers WHERE cust...,0.185022,18,1,0.010279,0.185022,97.29,5.40,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
4,### SYSTEM\nYou are a SQL query generation ass...,SELECT technician.Name,SELECT T3.Name FROM repair_assignment AS T1 JO...,0.185022,6,1,0.030837,0.185022,32.43,5.40,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35835,### SYSTEM\nYou are a SQL query generation ass...,SELECT Advisor FROM Student GROUP BY Advisor H...,SELECT Advisor FROM STUDENT GROUP BY Advisor H...,0.138727,23,1,0.006032,0.138727,165.79,7.21,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN
35836,### SYSTEM\nYou are a SQL query generation ass...,"SELECT state, SUM(acc_bal) as total_balance","SELECT sum(acc_bal) , state FROM customer WHE...",0.138727,15,1,0.009248,0.138727,108.13,7.21,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN
35837,### SYSTEM\nYou are a SQL query generation ass...,"SELECT artist_name, most_popular_in",SELECT artist_name FROM song WHERE resolution ...,0.138727,12,1,0.011561,0.138727,86.50,7.21,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN
35838,### SYSTEM\nYou are a SQL query generation ass...,"SELECT Employees.Employee_ID, COUNT(All_Docume...","SELECT Destroyed_by_Employee_ID , count(*) FR...",0.138727,33,2,0.004204,0.138727,237.88,14.42,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN


In [9]:
merged_batch_df['model'].unique()

array(['Mistral-7B-Instruct-v0.3', 'Qwen2.5-3B-Instruct',
       'Llama-3.1-8B-Instruct'], dtype=object)